In [1]:
import os
import pickle

import pandas as pd

from graphs import load_dataset_to_graph
from measurements import get_graph_measurements, compare_graph_measurements, \
    print_comparison_results
from recreate_graph import recreate_by_priority_rank_random_rankings, \
    recreate_by_priority_rank, get_trained_model

Using TensorFlow backend.
/home/smoq/.virtualenvs/cna/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
prepared_datasets_path = 'prepared_datasets'
prepared_dataframes_path = 'prepared_dataframes'
delimiter = '\t'

prepared_dataset_names = [
    'primary_school', 'workplace', 
    'highschool_2011', 'highschool_2012', 'hospital', 
    'moreno_blogs', 'moreno_sheep', 'moreno_seventh',
    # big datasets
    'petster-hamster', 'email-Eu'
]

comparison_dir = 'results'
number_of_comparisons = 10

In [3]:
# Random Rankings
print('Random rankings')
for dataset_name in prepared_dataset_names:
    dataset_path = os.path.join(prepared_datasets_path, dataset_name)
    dataframe_path = os.path.join(prepared_dataframes_path, dataset_name)
    graph = load_dataset_to_graph(dataset_path)
    
    random_rankings_list = []
    results_dir = os.path.join(comparison_dir, dataset_name)
    random_rankings_path = os.path.join(comparison_dir, dataset_name, 'random_rankings.pkl')
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    print('Processing {}'.format(dataset_name))

    for i in range(number_of_comparisons):
        print('Recreation number {}...'.format(i))
        new_graph = recreate_by_priority_rank_random_rankings(graph)

        graph_measurements = get_graph_measurements(graph)
        new_graph_measurements = get_graph_measurements(new_graph)
        comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)
        random_rankings_list.append(comparison)
    with open(random_rankings_path, 'wb') as f:
        pickle.dump(random_rankings_list, f)

Random rankings
Processing primary_school
Recreation number 0...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 1...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing workplace
Recreation number 0...
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 1...
Recreation number 2...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in sqrt
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing highschool_2011
Recreation number 0...
Recreation number 1...
Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing highschool_2012
Recreation number 0...
Recreation number 1...
Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing hospital
Recreation number 0...
Recreation number 1...
Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Cutting nodes up to 600
Processing moreno_blogs
Recreation number 0...
Cann

Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cutting nodes up to 600
Processing email-Eu
Recreation number 0...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 1...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 2...
Cannot compute average_shortest_path_length - Graph is not wea

In [4]:
# Only graph attributes
print('Only graph attributes')
for dataset_name in prepared_dataset_names:
    dataset_path = os.path.join(prepared_datasets_path, dataset_name)
    dataframe_path = os.path.join(prepared_dataframes_path, dataset_name)
    graph = load_dataset_to_graph(dataset_path)
    
    graph_attrs_list = []
    results_dir = os.path.join(comparison_dir, dataset_name)
    graph_attrs_path = os.path.join(comparison_dir, dataset_name, 'graph_attrs.pkl')
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    print('Processing {}'.format(dataset_name))
    graph_attrs_dataframe_path = os.path.join(dataframe_path, 'graph_attrs.csv')
    df = pd.read_csv(graph_attrs_dataframe_path, delimiter=delimiter)

    model = get_trained_model(df, epochs=16)
    # drop target column
    X_test = df.drop(['num_of_edges'], axis=1)
    # predict num_edges
    y_pred = model.predict(X_test)

    for i in range(number_of_comparisons):
        print('Recreation number {}...'.format(i))
        new_graph = recreate_by_priority_rank(graph, y_pred)

        graph_measurements = get_graph_measurements(graph)
        new_graph_measurements = get_graph_measurements(new_graph)
        comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)
        graph_attrs_list.append(comparison)
    with open(graph_attrs_path, 'wb') as f:
        pickle.dump(graph_attrs_list, f)

Only graph attributes
Processing primary_school
Epoch 1/16
58564/58564 [==============================] - 1s 14us/step - loss: 0.1579 - acc: 0.7154
Epoch 2/16
58564/58564 [==============================] - 1s 13us/step - loss: 0.0625 - acc: 0.7160
Epoch 3/16
58564/58564 [==============================] - 1s 14us/step - loss: 0.0285 - acc: 0.7160
Epoch 4/16
58564/58564 [==============================] - 1s 13us/step - loss: 0.0147 - acc: 0.7160
Epoch 5/16
58564/58564 [==============================] - 1s 14us/step - loss: 0.0083 - acc: 0.7160
Epoch 6/16
58564/58564 [==============================] - 1s 14us/step - loss: 0.0051 - acc: 0.7160
Epoch 7/16
58564/58564 [==============================] - 1s 14us/step - loss: 0.0033 - acc: 0.7160
Epoch 8/16
58564/58564 [==============================] - 1s 13us/step - loss: 0.0022 - acc: 0.7160
Epoch 9/16
58564/58564 [==============================] - 1s 14us/step - loss: 0.0016 - acc: 0.7160
Epoch 10/16
58564/58564 [===========================

/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 1...
Recreation number 2...
Recreation number 3...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 4...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in sqrt
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing workplace
Epoch 1/16
8464/8464 [==============================] - 0s 23us/step - loss: 0.2319 - acc: 0.8007
Epoch 2/16
8464/8464 [==============================] - 0s 16us/step - loss: 0.2018 - acc: 0.8216
Epoch 3/16
8464/8464 [==============================] - 0s 18us/step - loss: 0.1754 - acc: 0.8216
Epoch 4/16
8464/8464 [==============================] - 0s 16us/step - loss: 0.1524 - acc: 0.8216
Epoch 5/16
8464/8464 [==============================] - 0s 14us/step - loss: 0.1325 - acc: 0.8216
Epoch 6/16
8464/8464 [==============================] - 0s 13us/step - loss: 0.1153 - acc: 0.8216
Epoch 7/16
8464/8464 [==============================] - 0s 16us/step - loss: 0.1006 - acc: 0.8216
Epoch 8/16
8464/8464 [==============================] - 0s 14us/step - loss: 0.0880 - acc: 0.8216
Epoch 9/16
8464/8464 [==============================] - 0s 15us/step - loss: 0.0

360000/360000 [==============================] - 6s 18us/step - loss: 0.0100 - acc: 0.9888
Epoch 3/16
360000/360000 [==============================] - 6s 16us/step - loss: 0.0099 - acc: 0.9888
Epoch 4/16
360000/360000 [==============================] - 6s 16us/step - loss: 0.0099 - acc: 0.9888
Epoch 5/16
360000/360000 [==============================] - 6s 15us/step - loss: 0.0099 - acc: 0.9888
Epoch 6/16
360000/360000 [==============================] - 5s 15us/step - loss: 0.0099 - acc: 0.9888
Epoch 7/16
360000/360000 [==============================] - 5s 15us/step - loss: 0.0099 - acc: 0.9888
Epoch 8/16
360000/360000 [==============================] - 5s 14us/step - loss: 0.0098 - acc: 0.9888
Epoch 9/16
360000/360000 [==============================] - 5s 15us/step - loss: 0.0098 - acc: 0.9888
Epoch 10/16
360000/360000 [==============================] - 5s 15us/step - loss: 0.0098 - acc: 0.9888
Epoch 11/16
360000/360000 [==============================] - 5s 15us/step - loss: 0.0098 - a

841/841 [==============================] - 0s 16us/step - loss: 0.1823 - acc: 0.5529
Epoch 7/16
841/841 [==============================] - 0s 16us/step - loss: 0.1811 - acc: 0.5529
Epoch 8/16
841/841 [==============================] - 0s 22us/step - loss: 0.1799 - acc: 0.5529
Epoch 9/16
841/841 [==============================] - 0s 19us/step - loss: 0.1788 - acc: 0.5529
Epoch 10/16
841/841 [==============================] - 0s 14us/step - loss: 0.1778 - acc: 0.5529
Epoch 11/16
841/841 [==============================] - 0s 15us/step - loss: 0.1768 - acc: 0.5529
Epoch 12/16
841/841 [==============================] - 0s 20us/step - loss: 0.1758 - acc: 0.5529
Epoch 13/16
841/841 [==============================] - 0s 20us/step - loss: 0.1748 - acc: 0.5529
Epoch 14/16
841/841 [==============================] - 0s 14us/step - loss: 0.1738 - acc: 0.5529
Epoch 15/16
841/841 [==============================] - 0s 16us/step - loss: 0.1728 - acc: 0.5529
Epoch 16/16
841/841 [========================

Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 2...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 3...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 4...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Fou

In [5]:
# Only real attributes
print('Only real attributes')
for dataset_name in prepared_dataset_names:
    dataset_path = os.path.join(prepared_datasets_path, dataset_name)
    dataframe_path = os.path.join(prepared_dataframes_path, dataset_name)
    graph = load_dataset_to_graph(dataset_path)

    real_attrs_list = []
    results_dir = os.path.join(comparison_dir, dataset_name)
    real_attrs_path = os.path.join(comparison_dir, dataset_name, 'real_attrs.pkl')
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
        
    print('Processing {}'.format(dataset_name))
    real_attrs_dataframe_path = os.path.join(dataframe_path, 'real_attrs.csv')
    df = pd.read_csv(real_attrs_dataframe_path, delimiter=delimiter)

    model = get_trained_model(df, epochs=16)
    # drop target column
    X_test = df.drop(['num_of_edges'], axis=1)
    # predict num_edges
    y_pred = model.predict(X_test)

    for i in range(number_of_comparisons):
        print('Recreation number {}...'.format(i))
        new_graph = recreate_by_priority_rank(graph, y_pred)

        graph_measurements = get_graph_measurements(graph)
        new_graph_measurements = get_graph_measurements(new_graph)
        comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)
        real_attrs_list.append(comparison)
    with open(real_attrs_path, 'wb') as f:
        pickle.dump(real_attrs_list, f)

Only real attributes
Processing primary_school
Epoch 1/16
58564/58564 [==============================] - 1s 22us/step - loss: 0.0097 - acc: 0.7129
Epoch 2/16
58564/58564 [==============================] - 1s 15us/step - loss: 7.3691e-04 - acc: 0.7160
Epoch 3/16
58564/58564 [==============================] - 1s 16us/step - loss: 7.2409e-04 - acc: 0.7160
Epoch 4/16
58564/58564 [==============================] - 1s 17us/step - loss: 7.0913e-04 - acc: 0.7160
Epoch 5/16
58564/58564 [==============================] - 1s 16us/step - loss: 6.9113e-04 - acc: 0.7160
Epoch 6/16
58564/58564 [==============================] - 1s 15us/step - loss: 6.6050e-04 - acc: 0.7160
Epoch 7/16
58564/58564 [==============================] - 1s 15us/step - loss: 6.2484e-04 - acc: 0.7160
Epoch 8/16
58564/58564 [==============================] - 1s 16us/step - loss: 6.0176e-04 - acc: 0.7160
Epoch 9/16
58564/58564 [==============================] - 1s 17us/step - loss: 5.9460e-04 - acc: 0.7160
Epoch 10/16
58564/585

/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in sqrt
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 3...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing workplace
Epoch 1/16
8464/8464 [==============================] - 0s 47us/step - loss: 0.1561 - acc: 0.8216
Epoch 2/16
8464/8464 [==============================] - 0s 16us/step - loss: 0.0129 - acc: 0.8216
Epoch 3/16
8464/8464 [==============================] - 0s 17us/step - loss: 0.0013 - acc: 0.8216
Epoch 4/16
8464/8464 [==============================] - 0s 16us/step - loss: 7.7916e-04 - acc: 0.8216
Epoch 5/16
8464/8464 [==============================] - 0s 17us/step - loss: 6.5085e-04 - acc: 0.8216
Epoch 6/16
8464/8464 [==============================] - 0s 15us/step - loss: 6.0154e-04 - acc: 0.8216
Epoch 7/16
8464/8464 [==============================] - 0s 16us/step - loss: 5.7759e-04 - acc: 0.8216
Epoch 8/16
8464/8464 [==============================] - 0s 18us/step - loss: 5.6417e-04 - acc: 0.8216
Epoch 9/16
8464/8464 [===============

/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 3...
Recreation number 4...
Recreation number 5...
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 6...
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing highschool_2011
Epoch 1/16
15876/15876 [==============================] - 1s 32us/step - loss: 0.0850 - acc: 0.7320
Epoch 2/16
15876/15876 [==============================] - 0s 19us/step - loss: 0.0013 - acc: 0.7846
Epoch 3/16
15876/15876 [==============================] - 0s 15us/step - loss: 8.5566e-04 - acc: 0.7846
Epoch 4/16
15876/15876 [==============================] - 0s 16us/step - loss: 7.9221e-04 - acc: 0.7846
Epoch 5/16
15876/15876 [==============================] - 0s 14us/step - loss: 7.7220e-04 - acc: 0.7846
Epoch 6/16
15876/15876 [==============================] - 0s 14us/step - loss: 7.6405e-04 - acc: 0.784

Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 1...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 2...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 3...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Fou

360000/360000 [==============================] - 10s 27us/step - loss: 0.0051 - acc: 0.9948
Epoch 5/16
360000/360000 [==============================] - 10s 28us/step - loss: 0.0051 - acc: 0.9948
Epoch 6/16
360000/360000 [==============================] - 11s 29us/step - loss: 0.0051 - acc: 0.9948
Epoch 7/16
360000/360000 [==============================] - 11s 30us/step - loss: 0.0051 - acc: 0.9948
Epoch 8/16
360000/360000 [==============================] - 10s 29us/step - loss: 0.0051 - acc: 0.9948
Epoch 9/16
360000/360000 [==============================] - 11s 30us/step - loss: 0.0050 - acc: 0.9948
Epoch 10/16
360000/360000 [==============================] - 11s 30us/step - loss: 0.0050 - acc: 0.9948
Epoch 11/16
360000/360000 [==============================] - 10s 29us/step - loss: 0.0050 - acc: 0.9948
Epoch 12/16
360000/360000 [==============================] - 10s 29us/step - loss: 0.0050 - acc: 0.9948
Epoch 13/16
360000/360000 [==============================] - 10s 29us/step - loss

Recreation number 8...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 9...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected


In [6]:
round(len(df.drop(columns=['num_of_edges']).columns)/2)

42

In [7]:
# Both graph and real attributes
print('Both graph and real attributes')
for dataset_name in prepared_dataset_names:
    dataset_path = os.path.join(prepared_datasets_path, dataset_name)
    dataframe_path = os.path.join(prepared_dataframes_path, dataset_name)
    graph = load_dataset_to_graph(dataset_path)
    
    graph_real_attrs_list = []
    results_dir = os.path.join(comparison_dir, dataset_name)
    graph_real_attrs_path = os.path.join(comparison_dir, dataset_name, 'graph_real_attrs.pkl')
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
        
    print('Processing {}'.format(dataset_name))
    graph_real_attrs_dataframe_path = os.path.join(dataframe_path, 'graph_real_attrs.csv')
    df = pd.read_csv(graph_real_attrs_dataframe_path, delimiter=delimiter)

    model = get_trained_model(df, epochs=16)
    # drop target column
    X_test = df.drop(['num_of_edges'], axis=1)
    # predict num_edges
    y_pred = model.predict(X_test)

    for i in range(number_of_comparisons):
        print('Recreation number {}...'.format(i))
        new_graph = recreate_by_priority_rank(graph, y_pred)

        graph_measurements = get_graph_measurements(graph)
        new_graph_measurements = get_graph_measurements(new_graph)
        comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)
        graph_real_attrs_list.append(comparison)
    with open(graph_real_attrs_path, 'wb') as f:
        pickle.dump(graph_real_attrs_list, f)

Both graph and real attributes
Processing primary_school
Epoch 1/16
58564/58564 [==============================] - 1s 23us/step - loss: 0.0098 - acc: 0.7133
Epoch 2/16
58564/58564 [==============================] - 1s 16us/step - loss: 7.5777e-04 - acc: 0.7160
Epoch 3/16
58564/58564 [==============================] - 1s 16us/step - loss: 7.4593e-04 - acc: 0.7160
Epoch 4/16
58564/58564 [==============================] - 1s 16us/step - loss: 7.2786e-04 - acc: 0.7160
Epoch 5/16
58564/58564 [==============================] - 1s 16us/step - loss: 7.0175e-04 - acc: 0.7160
Epoch 6/16
58564/58564 [==============================] - 1s 16us/step - loss: 6.6013e-04 - acc: 0.7160
Epoch 7/16
58564/58564 [==============================] - 1s 16us/step - loss: 6.1640e-04 - acc: 0.7160
Epoch 8/16
58564/58564 [==============================] - 1s 16us/step - loss: 5.5891e-04 - acc: 0.7160
Epoch 9/16
58564/58564 [==============================] - 1s 16us/step - loss: 5.4103e-04 - acc: 0.7160
Epoch 10/16

/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in sqrt
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 1...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing workplace
Epoch 1/16
8464/8464 [==============================] - 1s 69us/step - loss: 0.0263 - acc: 0.8201
Epoch 2/16
8464/8464 [==============================] - 0s 16us/step - loss: 0.0010 - acc: 0.8216
Epoch 3/16
8464/8464 [==============================] - 0s 15us/step - loss: 6.4034e-04 - acc: 0.8216
Epoch 4/16
8464/8464 [==============================] - 0s 16us/step - loss: 5.8149e-04 - acc: 0.8216
Epoch 5/16
8464/8464 [==============================] - 0s 16us/step - loss: 5.6399e-04 - acc: 0.8216
Epoch 6/16
8464/8464 [==============================] - 0s 15us/step - loss: 5.5687e-04 - acc: 0.8216
Epoch 7/16
8464/8464 [==============================] - 0s 17us/step - loss: 5.5334e-04 - acc: 0.8216
Epoch 8/16
8464/8464 [==============================] - 0s 15us/step - loss: 5.5135e-04 -

/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing highschool_2012
Epoch 1/16
32400/32400 [==============================] - 1s 32us/step - loss: 0.0216 - acc: 0.8568
Epoch 2/16
32400/32400 [==============================] - 1s 16us/step - loss: 4.9837e-04 - acc: 0.8630
Epoch 3/16
32400/32400 [==============================] - 0s 15us/step - loss: 4.7438e-04 - acc: 0.8630
Epoch 4/16
32400/32400 [==============================] - 1s 16us/step - loss: 4.6908e-04 - acc: 0.8630
Epoch 5/16
32400/32400 [==============================] - 1s 16us/step - loss: 4.6719e-04 - acc: 0.8630
Epoch 6/16
32400/32400 [==============================] - 0s 15us/step - loss: 4.6635e-04 - acc: 0.8630
Epoch 7/16
32400/32400 [==============================] - 1s 16us/step - loss: 4.6593e-04 - acc: 0.8630
Epoch 8/16
32400/32400 [==============================] - 1s 16us/step - loss: 4.6569e-04 - acc: 0.8630
Epoch 9/16
32400/32400 [======

Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 9...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Processing moreno_sheep
Epoch 1/16
784/784 [==============================] - 1s 706us/step - loss: 0.2025 - acc: 0.6811
Epoch 2/16
784/784 [==============================] - 0s 16us/step - loss: 0.1917 - acc: 0.6811
Epoch 3/16
784/784 [==============================] - 0s 18us/step - loss: 0.1709 - acc: 0.6811
Epoch 4/16
784/784 [==============================] - 0s 17us/step - loss: 0.1471 - acc: 0.6811
Epoch 5/16
784/784

Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 4...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 5...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 6...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreat

In [8]:
# Real y
print('Real y')
for dataset_name in prepared_dataset_names:
    dataset_path = os.path.join(prepared_datasets_path, dataset_name)
    dataframe_path = os.path.join(prepared_dataframes_path, dataset_name)
    graph = load_dataset_to_graph(dataset_path)

    real_y_list = []
    results_dir = os.path.join(comparison_dir, dataset_name)
    real_y_path = os.path.join(comparison_dir, dataset_name, 'real_y.pkl')
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    print('Processing {}'.format(dataset_name))
    real_y_dataframe_path = os.path.join(dataframe_path, 'no_attrs.csv')
    df = pd.read_csv(real_y_dataframe_path, delimiter=delimiter)

    for i in range(number_of_comparisons):
        print('Recreation number {}...'.format(i))
        new_graph = recreate_by_priority_rank(graph, df['num_of_edges'].as_matrix())

        graph_measurements = get_graph_measurements(graph)
        new_graph_measurements = get_graph_measurements(new_graph)
        comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)
        real_y_list.append(comparison)
    with open(real_y_path, 'wb') as f:
        pickle.dump(real_y_list, f)

Real y
Processing primary_school
Recreation number 0...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 1...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in sqrt
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 2...
Recreation number 3...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing workplace
Recreation number 0...
Recreation number 1...
Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing highschool_2011
Recreation number 0...
Recreation number 1...
Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing highschool_2012
Recreation number 0...
Recreation number 1...
Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
Processing hospital
Recreation number 0...
Recreation number 1...
Recreation number 2...
Recreati

Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 8...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Recreation number 9...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cutting nodes up to 600
Processing email-Eu
Recreation number 0...
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because t